<a href="https://colab.research.google.com/github/fsilvao/ia_public/blob/main/Clase6_AE_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

%matplotlib inline
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


#Hiper parámetros
latent_dims= 10 # El tamaño de z
num_epochs=50
batch_size=128
capacity = 64
learning_rate = 1e-3
use_gpu= True

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = MNIST(root='./data/MNIST', download=True, train=True, transform=img_transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MNIST(root='./data/MNIST', download=True, train=False, transform=img_transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)




class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    c = capacity
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=c, kernel_size=4, stride=2, padding=1) # out: c x 14 x 14 , las imágenes de MNIS vienen en un canal, de 28 x 28
    self.conv2 = nn.Conv2d(in_channels= c, out_channels = c*2, kernel_size=4, stride=2, padding=1) #1.- Determinar el tamaño del output a pasar a la siguiente capa
    self.fc = #2.- Crear una capa lineal que tome como entrada la salida de la capa convolucional anterior, y como salida la variable que representa a z

  def forward(self,x):
    x=#3.- Pasar el dato x por la primera capa convolucional, luego aplicar la función de activación ReLU
    x= #4.- Pasar el dato x por la segunda capa convolucional, luego aplicar la función de activación ReLU
    x = x.view(x.size(0), -1) #Flatten para pasar a la capa fully connected
    x = self.fc(x)
    return x

class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    c = capacity
    self.fc = #5.- Invertimos las operaciones del codificador, partimos con la capa oculta. Determine el tamaño del input y el del output
    self.conv2 = nn.ConvTranspose2d(in_channels = c*2, out_channels=c, kernel_size=4, stride=2, padding=1)
    self.conv1 = nn.ConvTranspose2d(in_channels = c, out_channels=1, kernel_size=4, stride=2, padding=1)

  def forward(self,x):
    x=self.fc(x)
    x= x.view(x.size(0), capacity*2, 7, 7) #unflatten
    x= F.relu(self.conv2(x))
    x= torch.tanh(self.conv1(x))
    return x


class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder,self).__init__()
    self.encoder = # 6.- Instanciamos el codificador
    self.decoder = # 7.- Instanciamos el decodidicador

  def forward(self, x):
    latent = self.encoder(x)
    x_recon = #7.- Pasar la salida del encoder por el decodificador
    return x_recon

autoencoder = Autoencoder()

device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
autoencoder = autoencoder.to(device)


#Entrenamiento del autoencoder

optimizer = torch.optim.Adam(params=autoencoder.parameters(), lr=learning_rate, weight_decay = 1e-5)

#Setear para entrenar
autoencoder.train()

train_loss_avg = []

print("Entrenando...")
for epoch in range(num_epochs):
  train_loss_avg.append(0)
  num_batches=0

  for image_batch, _ in train_dataloader:

    image_batch = image_batch.to(device)

    #Reconstrucción del autoencoder
    image_batch_recon = autoencoder(image_batch)

    #Loss de reconstruccion
    loss = #8.- Cálculo del Loss

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()

    #Un paso del optimizer (usando los gradientes de backpropagation)
    optimizer.step()

    train_loss_avg[-1] += loss.item()
    num_batches +=1

  train_loss_avg[-1]/= num_batches
  print('Epoch [%d / %d] promedio de error de reconstruccion: %f' % (epoch+1, num_epochs, train_loss_avg[-1]))



import matplotlib.pyplot as plt
plt.ion()
fig = plt.figure()
plt.plot(train_loss_avg)
plt.xlabel('Epochs')
plt.ylabel('Error de reconstruccion')
plt.show()

#Evaluamos en el test set:

autoencoder.eval()

test_loss_avg, num_batches=0,00
for image_batch, _ in test_dataloader:
  with torch.no_grad():
    image_batch = image_batch.to(device)

    #reconstruccion del autoencoder
    image_batch_recon = autoencoder(image_batch)

    #error de reconstruccion:
    loss = #9.- Cálculo del Loss

    test_loss_avg += loss.item()
    num_batches +=1

test_loss_avg /= num_batches
print('Promedio de error de reconstruccion: %f' % (test_loss_avg))

import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import torchvision.utils

autoencoder.eval()

# Esta función toma como input las imagenes a reconstruir
# y el nombre del modelo con el cual realizar las reconstrucciones
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    return x

def show_image(img):
    img = to_img(img)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

def visualise_output(images, model):

    with torch.no_grad():

        images = images.to(device)
        images = model(images)
        images = images.cpu()
        images = to_img(images)
        np_imagegrid = torchvision.utils.make_grid(images[1:50], 10, 5).numpy()
        plt.imshow(np.transpose(np_imagegrid, (1, 2, 0)))
        plt.show()

images, labels = iter(test_dataloader).next()

# Visualizar las imágenes originales
print('Original images')
show_image(torchvision.utils.make_grid(images[1:50],10,5))
plt.show()

# Reconstruir y visualizar las imágenes usando el autoencoder entrenado
print('Autoencoder reconstruction:')
visualise_output(images, autoencoder)

Entrenando...
Epoch [1 / 50] promedio de error de reconstruccion: 0.116976
Epoch [2 / 50] promedio de error de reconstruccion: 0.070290
Epoch [3 / 50] promedio de error de reconstruccion: 0.063312
Epoch [4 / 50] promedio de error de reconstruccion: 0.059672
Epoch [5 / 50] promedio de error de reconstruccion: 0.057277
Epoch [6 / 50] promedio de error de reconstruccion: 0.055588
Epoch [7 / 50] promedio de error de reconstruccion: 0.054316
Epoch [8 / 50] promedio de error de reconstruccion: 0.053297
Epoch [9 / 50] promedio de error de reconstruccion: 0.052475
Epoch [10 / 50] promedio de error de reconstruccion: 0.051853
Epoch [11 / 50] promedio de error de reconstruccion: 0.051265
Epoch [12 / 50] promedio de error de reconstruccion: 0.050878
Epoch [13 / 50] promedio de error de reconstruccion: 0.050365
Epoch [14 / 50] promedio de error de reconstruccion: 0.050019
Epoch [15 / 50] promedio de error de reconstruccion: 0.049717


KeyboardInterrupt: 